<a href="https://colab.research.google.com/github/Tonge-Shim/1/blob/master/rfe%2Bforest_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import DataFrame
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier(max_iter=5, random_state=42)

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
df= pd.read_csv('samp_cst_feat.csv')
dtype = pd.read_excel('variable_dtype.xlsx')
dtype_categ = dtype[(dtype['dType']=='categorical')]
dtype_numrc = dtype[(dtype['dType']=='numerical')]
numrc_var = dtype_numrc['Variable_Name']
categ_var = dtype_categ['Variable_Name']
numrc_var = numrc_var.values.tolist()
categ_var = categ_var.values.tolist()#0,1
x=df.drop(categ_var, axis=1).drop(['cst_id_di'],axis=1)#10124개
y=pd.read_csv('samp_train.csv').drop(['cst_id_di'],axis=1)#10124
y_nozero= y[(y['MRC_ID_DI']!=0)]#not 0!!!1924
#y_nozeroindex=y_nozero.loc[1:10119]
y_nozeroindexlist=y_nozero.index.tolist()
x_nozero=x.loc[y_nozeroindexlist]

#rfe-feature selection

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

logreg=LogisticRegression()
selector=RFE(logreg)
selector=selector.fit(x_nozero,y_nozero)

In [ ]:
selected_features=DataFrame({'Feature': list(x_nozero.columns), 'Ranking': selector.ranking_})
selected_features.sort_values(by='Ranking').head(100)
#가장 연관성 높은 feature부터 나열한 자료

,Feature,Ranking
197,VAR227,1
88,VAR105,1
87,VAR104,1
86,VAR103,1
85,VAR102,1
...,...,...
177,VAR206,1
178,VAR207,1
179,VAR209,1
40,VAR046,1


In [ ]:
newx=selected_features.sort_values(by='Ranking')['Feature']
newx=newx.loc[197:40]#랭킹이 '1'이라고 표시된부분, 99개의 feature를 선택해서 리스트화
newx.tolist()
x_nozero=x_nozero[newx]#x_nozero는 자료와 연관성이 제일 높은 99개의 feature를 나타낸 자료, 
#참고로 이 자료는 '0'으로 분류된 클래스는 없음, shape는 (1924,99)

In [ ]:
print(x_nozero.shape)
print(y_nozero.shape)
#print(x.shape)
#print(y.shape)
y_nozeroarray=y_nozero.to_numpy()#1924
#y_array=y.to_numpy()#10124
x_nozeroarray=x_nozero.to_numpy()#1924
#x_array=x.to_numpy()#10124

(1924, 99)
(1924, 1)


#0을 제외한 1924개

In [ ]:
x_train, x_test, y_train, y_test=x_nozeroarray[:1400], x_nozeroarray[1400:], y_nozeroarray[:1400], y_nozeroarray[1400:]
shuffle_index=np.random.permutation(1400)
x_train, y_train=x_train[shuffle_index], y_train[shuffle_index]

#0기준 이진분류

In [ ]:
X_train, X_test, Y_train, Y_test=x_array[:9000], x_array[9000:], y_array[:9000], y_array[9000:]
shuffle_index=np.random.permutation(9000)
X_train, Y_train=X_train[shuffle_index], Y_train[shuffle_index]

Y_train_0=(Y_train==0)#0은 true, 0이 아니면 false
Y_test_0=(Y_test==0)



#0제외한 1924데이터 분류

In [ ]:
y_1=y[(y['MRC_ID_DI']==1)]
y_2=y[(y['MRC_ID_DI']==2)]
y_3=y[(y['MRC_ID_DI']==3)]
y_4=y[(y['MRC_ID_DI']==4)]
y_5=y[(y['MRC_ID_DI']==5)]
y_6=y[(y['MRC_ID_DI']==6)]
y_7=y[(y['MRC_ID_DI']==7)]
y_8=y[(y['MRC_ID_DI']==8)]
y_9=y[(y['MRC_ID_DI']==9)]
y_10=y[(y['MRC_ID_DI']==10)]

x_1=x_nozero.loc[y_1.index.tolist()]
x_2=x_nozero.loc[y_2.index.tolist()]
x_3=x_nozero.loc[y_3.index.tolist()]
x_4=x_nozero.loc[y_4.index.tolist()]
x_5=x_nozero.loc[y_5.index.tolist()]
x_6=x_nozero.loc[y_6.index.tolist()]
x_7=x_nozero.loc[y_7.index.tolist()]
x_8=x_nozero.loc[y_8.index.tolist()]
x_9=x_nozero.loc[y_9.index.tolist()]
x_10=x_nozero.loc[y_10.index.tolist()]

In [ ]:
y_nozero.value_counts()

MRC_ID_DI
7            661
6            554
8            337
5            143
1             85
9             59
3             32
10            28
2             14
4             11
dtype: int64

#5,6,7,8 분류 64%정확도(forest, highest)

In [ ]:
x_5678=x_nozero.loc[y_5.index.tolist()+y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()]
y_5678=y.loc[x_5678.index.tolist()]
print(x_5678.shape)
print(y_5678.shape)
x_5678=x_5678.to_numpy()
y_5678=y_5678.to_numpy()

(1695, 99)
(1695, 1)


In [ ]:
x_5678train, x_5678test, y_5678train, y_5678test=x_5678[:1250], x_5678[1250:], y_5678[:1250], y_5678[1250:]
shuffle_index=np.random.permutation(1250)
x_5678train, y_5678train=x_5678train[shuffle_index], y_5678train[shuffle_index]
some_digit=x_5678[3]#y_5678[3]=5

In [ ]:
sgd_clf.fit(x_5678train, y_5678train)
sgd_clf.predict([some_digit])

array([6])

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, x_5678train, y_5678train, cv=3, scoring="accuracy")

array([0.53956835, 0.59232614, 0.56730769])

In [ ]:
from sklearn.ensemble import RandomForestClassifier#제일 나음
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_5678train, y_5678train)
forest_clf.predict([some_digit])

array([5])

In [ ]:
cross_val_score(forest_clf, x_5678train, y_5678train, cv=3, scoring="accuracy")

array([0.64028777, 0.60431655, 0.60336538])

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
ovo_clf=OneVsOneClassifier(SGDClassifier(max_iter=5, random_state=42))
ovo_clf.fit(x_5678train, y_5678train)
ovo_clf.predict([some_digit])

array([5])

In [ ]:
cross_val_score(ovo_clf, x_5678train, y_5678train, cv=3, scoring="accuracy")

array([0.59952038, 0.54676259, 0.51923077])

이건 아니지이이이잉

#678 분류 67%정확도

In [ ]:
x_678=x_nozero.loc[y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()]
y_678=y.loc[x_678.index.tolist()]
print(x_678.shape)
print(y_678.shape)
x_678=x_678.to_numpy()
y_678=y_678.to_numpy()

(1552, 99)
(1552, 1)


In [ ]:
x_678train, x_678test, y_678train, y_678test=x_678[:1230], x_678[1230:], y_678[:1230], y_678[1230:]
shuffle_index=np.random.permutation(1230)
x_678train, y_678train=x_678train[shuffle_index], y_678train[shuffle_index]
some_digit1=x_678[3]#y_678[3]=6

In [ ]:
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_678train, y_678train)
forest_clf.predict([some_digit1])

array([6])

In [ ]:
cross_val_score(forest_clf, x_678train, y_678train, cv=3, scoring="accuracy")

array([0.67804878, 0.67560976, 0.67317073])

In [ ]:
y.loc[y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()].value_counts()

MRC_ID_DI
7            661
6            554
8            337
dtype: int64


661/(661+554+337)*100=43

554/(661+554+337)*100=35.7

337/(661+554+337)*100=21.7




#6,7분류 약 70%정확도최고 72%


In [ ]:
x_67=x_nozero.loc[y_6.index.tolist()+y_7.index.tolist()]
y_67=y.loc[x_67.index.tolist()]
print(x_67.shape)
print(y_67.shape)
x_67=x_67.to_numpy()
y_67=y_67.to_numpy()

(1215, 99)
(1215, 1)


In [ ]:
x_67train, x_67test, y_67train, y_67test=x_67[:950], x_67[950:], y_67[:950], y_67[950:]
shuffle_index=np.random.permutation(950)
x_67train, y_67train=x_67train[shuffle_index], y_67train[shuffle_index]
some_digit1=x_67[3]#y_67[3]=6

In [ ]:
#randomforest
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_67train, y_67train)
forest_clf.predict([some_digit1])

array([6])

In [ ]:
cross_val_score(forest_clf, x_67train, y_67train, cv=3, scoring="accuracy")


array([0.65615142, 0.69716088, 0.7278481 ])

In [ ]:
#sgd_clf
sgd_clf.fit(x_67train, y_67train)
sgd_clf.predict([some_digit1])

array([6])

In [ ]:
cross_val_score(sgd_clf, x_67train, y_67train, cv=3, scoring="accuracy")

array([0.64037855, 0.6466877 , 0.68037975])

In [ ]:
y_nozero.value_counts()

MRC_ID_DI
7            661
6            554
8            337
5            143
1             85
9             59
3             32
10            28
2             14
4             11
dtype: int64

#2,3,4,9,10 약 60%정확도

In [ ]:
x_210=x_nozero.loc[y_2.index.tolist()+y_3.index.tolist()+y_4.index.tolist()+y_9.index.tolist()+y_10.index.tolist()]
y_210=y.loc[x_210.index.tolist()]
print(x_210.shape)
print(y_210.shape)
x_210=x_210.to_numpy()
y_210=y_210.to_numpy()

x_210train, x_210test, y_210train, y_210test=x_210[:110], x_210[110:], y_210[:110], y_210[110:]
shuffle_index=np.random.permutation(110)
x_210train, y_210train=x_210train[shuffle_index], y_210train[shuffle_index]
some_digit1=x_210[3]#y_210[3]=2

(144, 99)
(144, 1)


In [ ]:
#randomforest
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_210train, y_210train)
forest_clf.predict([some_digit1])

array([2])

In [ ]:
cross_val_score(forest_clf, x_210train, y_210train, cv=3, scoring="accuracy")


array([0.62162162, 0.51351351, 0.58333333])

In [ ]:
#sgd_clf
sgd_clf.fit(x_210train, y_210train)
sgd_clf.predict([some_digit1])
cross_val_score(sgd_clf, x_210train, y_210train, cv=3, scoring="accuracy")


array([0.67567568, 0.51351351, 0.36111111])

#rfe를 추가한 결과, 조금씩 높아지긴 했지만... 아직도 낮음